In [120]:
import sys
import ast
import pandas as pd
import numpy as np

In [121]:
listOfEvents = []
input_file = "out2.txt"
with open(input_file,"r") as f:
    for line in f:
        listOfEvents.append(ast.literal_eval(line))
df = pd.DataFrame.from_dict(listOfEvents)
# df["ReadAddr"]=""
# df["WriteAddr"]=""
df["AllocatedPages"]=""
# df["AllocatedIDs"]=""



In [122]:

df.dtypes

starttime           int64
address            object
size               object
type               object
cudatime           object
cudatimeoverlap    object
eventid             int64
AllocatedPages     object
dtype: object

In [123]:
# df.loc[df["kernelid"]==0,["ReadAddr"]]="anything"
# df[df["kernelid"]==0]["ReadAddr"]

In [124]:
nvbit_file = "../mem_trace_multilayer.txt"
with open(nvbit_file,"r") as f:
    for line in f:
        if "Read" in line:
            data = line.split(":")[1]
            data = data.split(",")
            kernelid = int(data[1])
            readAddr = [e.strip() for e in data[2:] if e != "0x0"]
            #print(readAddr)
            df.loc[(df["type"]!="allocate") & (df["eventid"]==kernelid),["ReadAddr"]]=str(readAddr)
        if "Write" in line:
            data = line.split(":")[1]
            data = data.split(",")
            kernelid = int(data[1])
            writeAddr = [e.strip() for e in data[2:] if e != "0x0"]
            #print(writeAddr)
            df.loc[(df["type"]!="allocate") & (df["eventid"]==kernelid),["WriteAddr"]]=str(writeAddr)
            


In [125]:
listOfEvents = []
input_file_nvbit = "out2_nvbit.txt"
with open(input_file_nvbit,"r") as f:
    for line in f:
        listOfEvents.append(ast.literal_eval(line))
df_realaddr = pd.DataFrame.from_dict(listOfEvents)

In [126]:
realaddr = df_realaddr["address"]
for i, ra in enumerate(realaddr):
    df.loc[(df["type"]=="allocate") & (df["eventid"] == i),["address"]]=ra

In [127]:
def get_pagesAllocated(addr, size):
    if size < 0:
        size = 0-size
    
    pages_to_return = []
    maxAddr = addr + size
    pageAddr = addr // 4096 *4096
    while pageAddr <= maxAddr:
        pages_to_return.append(hex(pageAddr))
        pageAddr += 4096
    return pages_to_return

In [128]:
df_realaddr['address'] = df_realaddr['address'].apply(lambda x:int(x,0))
df_realaddr['size'] = df_realaddr['size'].astype('int')
df_realaddr[['address','size']]


,address,size
0,1476130688,8
1,1476227904,8
2,1460523904,400000
3,1476230848,8
4,1476230848,-8
...,...,...
77,140133984632832,2048
78,140133961826304,1644032
79,140133961826304,-1644032
80,140136456139264,512


In [129]:
df_realaddr['col_3'] = df_realaddr.apply(lambda x: str(get_pagesAllocated(x['address'], x['size'])), axis=1)

In [130]:
df_realaddr['col_3'].isnull().sum()

0

In [131]:
pageaddr = df_realaddr["col_3"]
for i, pa in enumerate(pageaddr):
    df.loc[(df["type"]=="allocate") & (df["eventid"] == i),["AllocatedPages"]]=pa

In [132]:
df.to_csv(input_file.split('.')[0]+"2.csv")

In [145]:
mapping = {} # bufferid -> addr
curid = 0
addr_set = set()
for index, row in df.iterrows():
    if (row['type'] == "allocate"):
        #print(index, ',',row['AllocatedPages'], row['size'])
        page_list = ast.literal_eval(row['AllocatedPages'])
        local_mapping = {}
        print("size = ",row['size'])
        for p in page_list:
            if p[:4] == "0x7f":
                if p in addr_set:
                    if int(row['size']) < 0: #free
                        addr_set.remove(p) # does not modify curid, curid always increase even if free
                    else:#no free but allocate at the same address
                        addr_set.add(p)
                        mapping[curid]=p
                        local_mapping[curid]=p
                        curid += 1
                else:
                    addr_set.add(p)
                    mapping[curid]=p
                    local_mapping[curid]=p
                    curid += 1

        df.loc[index,'AllocatedIDs']=str(local_mapping)

size =  8
size =  8
size =  400000
size =  8
size =  -8
size =  393216
size =  512
size =  602112
size =  602112
size =  150528
size =  301056
size =  204800
size =  365056
size =  -365056
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
size =  204800
size =  51200
size =  102400
size =  15360
size =  1048576
size =  8519680
size =  -1048576
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am -
I am

In [146]:
df.loc[0:10,'AllocatedIDs']

0                                                    {}
1                                                    {}
2                                                    {}
3                                                    {}
4                                                    {}
5     {0: '0x7f740fa3d000', 1: '0x7f740fa3e000', 2: ...
6                                {97: '0x7f740fa9d000'}
7     {98: '0x7f740fa9d000', 99: '0x7f740fa9e000', 1...
8                                                   NaN
9                                                   NaN
10    {246: '0x7f740fb30000', 247: '0x7f740fb31000',...
Name: AllocatedIDs, dtype: object

In [ ]:
len(mapping)

4343

In [147]:
df.to_csv(input_file.split('.')[0]+"3.csv")